In [5]:
# import libraries
import os
#import os.path
from bs4 import BeautifulSoup
import requests
#import pandas as pd
import numpy as np
#import re
import csv
from csv import DictWriter
#import pathlib

### 1: לאסוף לינקים מ-4 עמודי חיפוש

In [12]:
''' --------------- search_page_handler(url, headers) -----------------------
gets: url=the url of the prodoct, headers=the thing needed to get amazon html
return: returns next page url and urls of the prodocts that whore in this search page
goes over the urls in the search page
'''
def put_urls_in_csv_from_search_pages(url, csv_name):
    page = requests.get(url)
    
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    # get prodocts urls
    urls = []
    for h in soup2.find_all("a"):
        url_string = h.attrs['href']
        sub = "./snorna_db.cgi?mode=sno_info"
        if(url_string.find(sub) != -1): #if 0 != -1 it's in the string
            urls.append('http://snoopy.med.miyazaki-u.ac.jp/' + url_string)
    #print(urls)
    
    with open(csv_name, 'a', encoding='UTF8') as f_object:
            # Pass this file object to csv.writer() and get a writer object
            writer_obj = csv.writer(f_object)
            # Pass the list as an argument into the writerow()
            writer_obj.writerow(urls)
            # Close the file object
            f_object.close()  
            

In [13]:
# URLS
url_RA_Mus_musculus = "http://snoopy.med.miyazaki-u.ac.jp/snorna_db.cgi?mode=sno_search&organism=Mus_musculus&mod_type=all&mod_rna=all&organization=all&keyword=SNORA&dataquality=all"
url_RA_Homo_sapiens = "http://snoopy.med.miyazaki-u.ac.jp/snorna_db.cgi?mode=sno_search&organism=Homo_sapiens&mod_type=all&mod_rna=all&organization=all&keyword=SNORA&dataquality=all"
url_RD_Mus_musculus = "http://snoopy.med.miyazaki-u.ac.jp/snorna_db.cgi?mode=sno_search&organism=Mus_musculus&mod_type=all&mod_rna=all&organization=all&keyword=SNORD&dataquality=all"
url_RD_Homo_sapiens = "http://snoopy.med.miyazaki-u.ac.jp/snorna_db.cgi?mode=sno_search&organism=Homo_sapiens&mod_type=all&mod_rna=all&organization=all&keyword=SNORD&dataquality=all"
# ------------------------------------------------------------------------------------------------------------------------------------_ ^ _--------------------------------------------------------------------------------------_ ^ _

# create CSV, for SNOXX url
csv_name1RA = 'urls_RA_Mus_musculus.csv'
csv_name2RA = 'urls_RA_Homo_sapiens.csv'
csv_name3RD = 'urls_RD_Mus_musculus.csv'
csv_name4RD = 'urls_RD_Homo_sapiens.csv'

with open(csv_name1RA, "w") as my_empty_csv:
    # now you have an empty file
    pass

put_urls_in_csv_from_search_pages(url_RA_Mus_musculus, csv_name1RA)


In [166]:
with open(csv_name2RA, "w") as my_empty_csv:
    # now you have an empty file
    pass
put_urls_in_csv_from_search_pages(url_RA_Homo_sapiens, csv_name2RA)

In [167]:
with open(csv_name3RD, "w") as my_empty_csv:
    # now you have an empty file
    pass
put_urls_in_csv_from_search_pages(url_RD_Mus_musculus, csv_name3RD)

In [168]:
with open(csv_name4RD, "w") as my_empty_csv:
    # now you have an empty file
    pass
put_urls_in_csv_from_search_pages(url_RD_Homo_sapiens, csv_name4RD)

### 2: 
על כל דף חיפוש.
לעבור על לינקים ולאסוף מידע, כל לינק שורה משלה

In [163]:
def scrap_info_from_urls_lst(lst_urls, csv_name, trait_dict):
    # trait_dict.keys() = ['Name', 'Sequence', 'Modification_Type', 'Url']
    #lst_urls = [lst_urls[0]] #for testing
    conter = 0
    total = 4
    status_forcelist = [429, 500, 502, 503, 504]
    lst_url_fail = []
    
    # going on every page on by one
    for current_url in lst_urls:
        # know on which url we are on
        print(conter)
        conter = conter + 1
        
        # requests and retry incase banned
        for _ in range(total):
            try:
                page = requests.get(current_url)
                if page.status_code in status_forcelist:
                    # Retry request 
                    continue
                else:
                    # scesis in requests, can stop now
                    break
            except requests.exceptions.ConnectionError:
                pass
        #page = requests.get(current_url)
        
        #check if 'page' is good, if not add to lst_url_fail for retry later
        if not page.ok:
            print('not page.ok')
            lst_url_fail.append(current_url)
            continue # to next url in lst_urls
        
        soup1 = BeautifulSoup(page.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

        # URL
        trait_dict['Url'] = current_url
        
        # Name
        for element in soup2.find_all('td', class_ = "info_index"):
            str_name = element.text
            str_name = str_name.replace('\n    ', '')
            trait_dict['Name'] = str_name
        
        # Sequence
        for element in soup2.find_all('td', class_ = "sequence"):
            sequence_info = str(element.contents[1]).replace('<br/>', '')
            #print(sequence_info)
            sequence_info = sequence_info.replace('\n', '')
            sequence_info = sequence_info.replace(' ', '')
            sequence_info = sequence_info.replace('<div>', '')
            sequence_info = sequence_info.replace('</div>', '')
            sequence_info = sequence_info.replace('spanclass=', '')
            sequence_info = sequence_info.replace('</span', '')
            #print(sequence_info)
            #print('--------')
            #sequence_info = sequence_info.replace('>', '> ')
            #sequence_info = sequence_info.replace('<', ' <')
            #print(sequence_info)
            trait_dict['Sequence'] = sequence_info
        
        
        # Modification_Type - modification type
        for element in soup2.find_all('td', class_ = "info_col"):
            str_trait_name = element.text
            sub = 'modification type'
            if(str_trait_name.find(sub) != -1):
                M_type_text = element.parent.contents[3].text
                M_type_text = M_type_text.replace('\n    ', '')
                trait_dict['Modification_Type'] = M_type_text
        
        # put prodoct in csv as new row
        # Open our existing CSV file in append mode
        with open(csv_name, 'a', encoding='UTF8') as f_object:
            field_names = trait_dict.keys()
            dictwriter_object = DictWriter(f_object, fieldnames = field_names)
            # Pass the dictionary as an argument to the Writerow()
            dictwriter_object.writerow(trait_dict)
            # Close the file object
            f_object.close()
        
        #print('--------')
        #print(trait_dict)
        
        # clean 'trait_dict' for next page
        trait_dict = dict.fromkeys(trait_dict, None)
        
    return lst_url_fail

In [164]:
def create_csvInfo(traits_cols_dict, csv_name_for_info, csv_name_for_urls):
    # create CSV_info
    lst_columns = traits_cols_dict.keys()
    with open(csv_name_for_info, "w") as my_empty_csv:
        # create the csv writer
        writer = csv.writer(my_empty_csv)
        # write a row to the csv file
        writer.writerow(lst_columns)
        my_empty_csv.close()
        pass

    lst_urls_temp = []
    # get list of urls from csv
    with open(csv_name_for_urls) as file_obj: 
        # Create reader object by passing the file  
        # object to reader method 
        reader_obj = csv.reader(file_obj) 
        # Iterate over each row in the csv  
        # file using reader object 
        for row in reader_obj: 
            lst_urls_temp.append(row)

    lst_urls = lst_urls_temp[0] 
    
    lst_url_fail = []
    lst_url_fail = scrap_info_from_urls_lst(lst_urls, csv_name_for_info, traits_cols_dict)
    
    while len(lst_url_fail) != 0: #lst_url_fail not empty:
        print('lst_url_fail not empty.')
        lst_url_fail = scrap_info_from_urls_lst(lst_url_fail, csv_name_for_info, traits_cols_dict)

In [165]:
basic_traits = ['Name', 'Sequence', 'Modification_Type', 'Url']
traits_cols_dict = dict.fromkeys(basic_traits, None)

# CSV names for SNOXX url
csv_name1RA_info = 'All_Info_RA_Mus_musculus.csv'
csv_name2RA_info = 'All_Info_RA_Homo_sapiens.csv'
csv_name3RD_info = 'All_Info_RD_Mus_musculus.csv'
csv_name4RD_info = 'All_Info_RD_Homo_sapiens.csv'

create_csvInfo(traits_cols_dict, csv_name1RA_info, csv_name1RA)
print('Done.') #642

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [169]:
create_csvInfo(traits_cols_dict, csv_name2RA_info, csv_name2RA)
print('Done.') #319

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [170]:
create_csvInfo(traits_cols_dict, csv_name3RD_info, csv_name3RD)
print('Done.') #324

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [171]:
create_csvInfo(traits_cols_dict, csv_name4RD_info, csv_name4RD)
print('Done.') #416

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27